In [ ]:
# account to access namespaces and pods list and details
kubectl create serviceaccount cp-api-explorer
# ro permissions
kubectl create clusterrole ns-reader --verb=get,list,watch --resource=namespaces,pods
# bind role(permissions) to account
kubectl create clusterrolebinding ns-reader --clusterrole ns-reader --serviceaccount default:cp-api-explorer


serviceaccount/cp-api-explorer created
clusterrole.rbac.authorization.k8s.io/ns-reader created
clusterrolebinding.rbac.authorization.k8s.io/ns-reader created


[doc](https://nieldw.medium.com/curling-the-kubernetes-api-server-d7675cfc398c)

In [ ]:
# account has secret(s)
kubectl get serviceaccount cp-api-explorer -o json

{
    "apiVersion": "v1",
    "kind": "ServiceAccount",
    "metadata": {
        "creationTimestamp": "2022-03-30T14:40:37Z",
        "name": "cp-api-explorer",
        "namespace": "default",
        "resourceVersion": "477",
        "uid": "c97ecfe0-0a61-4b7a-9bd6-9001ad0130cf"
    },
    "secrets": [
        {
            "name": "cp-api-explorer-token-w2m26"
        }
    ]
}


In [ ]:
# list secrets
kubectl get secrets

NAME                          TYPE                                  DATA   AGE
cp-api-explorer-token-w2m26   kubernetes.io/service-account-token   3      7s
default-token-cfl2r           kubernetes.io/service-account-token   3      16s


In [ ]:
# we know which from SA, right
$serviceAccount = "cp-api-explorer"
# get right secret
$secretName = kubectl get serviceaccount cp-api-explorer -o json | ConvertFrom-Json `
    | % { $_.secrets } `
    | ? { $_.name.startsWith("$($serviceAccount)-token-") } `
    | select -First 1 | % { $_.name }
$secretName
# secret from secretName
$token = kubectl get secret $secretName -o json | ConvertFrom-Json | % { $_.data.token | base64 -d }
# ca.crt
$caCrtFilename = "./k8sca.crt"
kubectl get secret $secretName -o json | ConvertFrom-Json | % { $_.data."ca.crt" | base64 -d } | Out-File $caCrtFilename
#gc $caCrtFilename

cp-api-explorer-token-w2m26


In [ ]:
# k8s API endpoint
$server = kubectl -n default get endpoints kubernetes --no-headers | awk '{ print $2 }'
$apiServerUrl = "https://$($server)"
$apiServerUrl

https://192.168.64.5:8443


In [ ]:
# try request
curl -s "$apiServerUrl/openapi/v2"  --header "Authorization: Bearer $token" --cacert $caCrtFilename

In [ ]:
/usr/bin/curl -v "$apiServerUrl/api/v1/namespaces/default/pods" `
   --header "Authorization: Bearer $token" --cacert $caCrtFilename

In [ ]:
$apiServerUrl="https://192.168.64.5:8443"

/usr/bin/curl -s "$apiServerUrl/api/v1/namespaces/kube-system/pods" `
   --header "Authorization: Bearer $token" --cacert $caCrtFilename | jq .

{
  "kind": "PodList",
  "apiVersion": "v1",
  "metadata": {
    "resourceVersion": "540"
  },
  "items": [
    {
      "metadata": {
        "name": "coredns-64897985d-cvmhb",
        "generateName": "coredns-64897985d-",
        "namespace": "kube-system",
        "uid": "6ea08388-66bd-4b30-8d7f-c1df0ee31cca",
        "resourceVersion": "465",
        "creationTimestamp": "2022-03-30T14:40:28Z",
        "labels": {
          "k8s-app": "kube-dns",
          "pod-template-hash": "64897985d"
        },
        "ownerReferences": [
          {
            "apiVersion": "apps/v1",
            "kind": "ReplicaSet",
            "name": "coredns-64897985d",
            "uid": "8f5b8f0d-bd75-4eb5-aed7-da31449c50b4",
            "controller": true,
            "blockOwnerDeletion": true
          }
        ],
        "managedFields": [
          {
            "manager": "kube-controller-manager",
            "operation": "Update",
            "apiVersion": "v1",
            "time": "2022-03-3

In [ ]:
kubectl config view


apiVersion: v1
clusters:
- cluster:
    certificate-authority-data: DATA+OMITTED
    server: https://api.crc.testing:6443
  name: api-crc-testing:6443
- cluster:
    certificate-authority: /Users/mkoldov/.minikube/ca.crt
    extensions:
    - extension:
        last-update: Wed, 30 Mar 2022 16:40:16 CEST
        provider: minikube.sigs.k8s.io
        version: v1.25.1
      name: cluster_info
    server: https://192.168.64.5:8443
  name: minikube
contexts:
- context:
    cluster: api-crc-testing:6443
    user: developer/api-crc-testing:6443
  name: /api-crc-testing:6443/developer
- context:
    cluster: api-crc-testing:6443
    namespace: app1
    user: developer/api-crc-testing:6443
  name: app1/api-crc-testing:6443/developer
- context:
    cluster: api-crc-testing:6443
    namespace: app1
    user: kubeadmin/api-crc-testing:6443
  name: app1/api-crc-testing:6443/kubeadmin
- context:
    cluster: api-crc-testing:6443
    namespace: default
    user: kubeadmin
  name: crc-admin
- contex